In [4]:
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
%matplotlib inline
from scipy import optimize

from tyssue.core.sheet import Sheet
from tyssue.config.json_parser import load_default
from tyssue.core.objects import get_opposite
from tyssue.core.objects import Epithelium

from tyssue.geometry.sheet_geometry import SheetGeometry as geom
from tyssue.dynamics.apoptosis_model import SheetApoptosisModel as model
from tyssue.dynamics.sheet_vertex_model import SheetModel as basemodel
from tyssue.solvers.sheet_vertex_solver import Solver as solver

#from tyssue.dynamics.sheet_isotropic_model import isotropic_relax

from tyssue.draw.vispy_draw import vp_view
from tyssue.topology.sheet_topology import type1_transition, remove_face

from tyssue.draw.plt_draw import sheet_view
import tyssue.draw.plt_draw as draw
from tyssue.io.hdf5 import load_datasets

In [5]:


specs = {
    'edge': {
        'dx': 0.0, 'dy': 0.0, 'dz': 0.0,
        'length': 0.0,
        'nx': 0.0, 'ny': 0.0, 'nz': 1.0,
        'srce': 0, 'trgt': 0, 'face': 0, 'cell': 0,
        'segment': 'apical',
        'sub_area': 0.0,
        'line_tension': 0.12,
        'basal_elasticity': 1.0,
        },
    'face': {
        'area': 1.0,
        'is_alive': True,
        'num_sides': 6,
        'perimeter': 1.0,
        'x': 0.0, 'y': 0.0, 'z': 0.0,
        'contractility': 0.04,
        'segement': 'apical'
        },
    'cell': {
        'vol': 1.0,
        'vol_elasticity': 1.0,
        'prefered_vol': 1.0,
        #'ab_elasticity': 1.0,
        #'prefered_hab': 1.0,
        'is_alive': True,
        'num_sides': 6,
        'perimeter': 1.0,
        'x': 0.0, 'y': 0.0, 'z': 0.0,
        },
     'settings': {
        'geometry': 'cylindrical',
        'height_axis': 'z'
        },
     'vert': {
        'is_active': True,
        'segment': 'apical',
        'x': 0.0, 'y': 0.0, 'z': 0.0,
        }
    }


In [7]:
ls

after.obj                     Fold formation - Sheet.ipynb  render.blend
before_apoptosis_anchors.hf5  Leg_joint_monolayer.ipynb     render.blend1
before_apoptosis.hf5          leg_joint_morphogenesis.py    render.stl
before_apoptosis.hf5.hf5      Mixed Geometry.ipynb          small_hexagonal.hf5
before_monolayer              Monolayer apoptosis.ipynb     specs.json
before.png                    Parameter exploration.ipynb


In [6]:
h5store = ''
datasets = load_datasets(h5store,
                         data_names=['face', 'vert', 'edge'])


coords = ['x', 'y', 'z']


### Create cells
datasets['cell'] = pd.DataFrame(index=datasets['face'].index,
                                columns=coords)
datasets['cell'].index.name = 'cell'
datasets['cell']['z'] = datasets['face']['z']
theta = np.arctan2(datasets['face']['y'],
                   datasets['face']['x']).values

rho = np.hypot(datasets['face']['y'],
               datasets['face']['x']).values
datasets['cell']['x'] = 2*rho/3 * np.cos(theta)
datasets['cell']['y'] = 2*rho/3 * np.sin(theta)


n_verts = datasets['vert'].shape[0]
n_cells = datasets['cell'].shape[0]

### Basal vertices
basal_idx = pd.Index(np.arange(n_verts, 2*n_verts),
                     name='vert')
basal_verts = pd.DataFrame(index=basal_idx, 
                           columns=datasets['vert'].columns)
                     
theta_v = np.arctan2(datasets['vert']['y'],
                   datasets['vert']['x']).values

rho_v = np.hypot(datasets['vert']['y'],
               datasets['vert']['x']).values
    
basal_verts['z'] = datasets['vert']['z'].values
basal_verts['x'] = rho_v/3 * np.cos(theta_v)
basal_verts['y'] = rho_v/3 * np.sin(theta_v)
basal_verts['is_active'] = 1
basal_verts['segment'] = 'basal'

datasets['vert'] = pd.concat([datasets['vert'],
                              basal_verts])

### Basal faces


datasets['edge']['cell'] = datasets['edge']['face']

opposite = get_opposite(datasets['edge'])

opp_face = datasets['edge'].loc[opposite[opposite>=0], 'face'].values
opp_cell = datasets['edge'].loc[opposite[opposite>=0], 'cell'].values

basal_edge_idx = pd.Index(
    np.arange(datasets['edge'].shape[0]) +
    datasets['edge'].index[-1], name='edge')

basal_edges = pd.DataFrame(index=basal_edge_idx,
                           columns=['srce', 'trgt', 'is_basal'])

basal_edges['srce'] = datasets['edge']['cell'].values + n_verts
basal_edges['trgt'] = opp_cell + n_verts
basal_edges['segment'] = 'basal'

datasets['edge'] = pd.concat([datasets['edge'], basal_edges])




KeyError: 'No object named face in the file'

In [2]:
eptm = Epithelium('mixed', datasets, specs)

NameError: name 'datasets' is not defined

In [9]:
from tyssue.geometry.sheet_geometry import SheetGeometry

class MixedGeometry(SheetGeometry):
    
    
    @classmethod
    def update_all(cls, eptm):
        '''
        Updates the epithelium geometry by updating:
        * the edge vector coordinates
        * the edge lengths
        * the normals to each edge associated face
        * the face areas
        * the vertices heights (depends on geometry)
        * the face volumes (depends on geometry)

        '''
        cls.update_dcoords(eptm)
        cls.update_length(eptm)
        cls.update_height(eptm)
        cls.update_normals(eptm)
        cls.update_areas(eptm)
        cls.update_perimeters(eptm)
        cls.update_centroid(eptm)
        cls.update_vol(eptm)
    
    @staticmethod
    def update_centroid(eptm):
        
        upcast_pos = eptm.upcast_srce(eptm.vert_df[sheet.coords])
        upcast_pos = upcast_pos.set_index(eptm.edge_df['face'], append=True)
        eptm.face_df[eptm.coords] = upcast_pos.mean(level='face')

    
    @staticmethod
    def update_vol(eptm):
        
        face_pos = eptm.upcast_face(
            eptm.face_df[eptm.coords])
        cell_pos = eptm.upcast_cell(
            eptm.cell_df[eptm.coords])
        
        eptm.edge_df['sub_vol'] = np.sum(
            (face_pos - cell_pos) *
            eptm.edge_df[eptm.ncoords].values, axis=1) / 6
        eptm.edge_df['sub_vol'] = eptm.edge_df['sub_vol'] * (
            1 - eptm.edge_df['is_basal'])
        eptm.cell_df['vol'] = eptm.sum_cell(eptm.edge_df['sub_vol'])


In [10]:
MixedGeometry.update_all(eptm)

In [14]:
from tyssue.dynamics.effectors import elastic_force, elastic_energy



class MixedModel():
    
    @staticmethod
    def dimentionalize(mod_specs):
        """
        Changes the values of the input gamma and lambda parameters
        from the values of the prefered height and area.
        Computes the norm factor.
        """

        dim_mod_specs = deepcopy(mod_specs)

        Kv = dim_mod_specs['cell']['vol_elasticity']
        V0 = dim_mod_specs['cell']['prefered_vol']
        gamma = dim_mod_specs['face']['contractility']
        kappa_ab = dim_mod_specs['cell']['ab_elasticity']
        kappa_bl = dim_mod_specs['edge']['basal_elasticity']
        lbda = dim_mod_specs['edge']['line_tension']
        
        dim_mod_specs['face']['contractility'] = gamma * Kv*V0
        dim_mod_specs['cell']['ab_elasticity'] = kappa_ab * Kv*V0
        dim_mod_specs['cell']['bl_elasticity'] = kappa_bl * Kv*V0

        dim_mod_specs['edge']['line_tension'] = lbda * Kv * V0**(5/3.)

        dim_mod_specs['settings']['grad_norm_factor'] = Kv * V0**(5/3.)
        dim_mod_specs['settings']['nrj_norm_factor'] = Kv * V0**2

        return dim_mod_specs

    @staticmethod
    def compute_energy(eptm):
        
        live_cells = eptm.cell_df[eptm.cell_df['is_alive']==1]
        apical_edges = eptm.edge_df[eptm.edge_df['is_basal' == 0]]
        E_t = apical_edges.eval('line_tension * length/2')
        E_v = elastic_energy(live_cells, var='vol',
                             elasticity='vol_elasticity',
                             prefered='prefered_vol')
        E_c = eptm.face_df.eval(
            'is_alive * 0.5 * contractility * perimeter ** 2')
        
        cell_pos = eptm.cell_df[eptm.coords].values
        bvert_pos = eptm.vert_df[eptm.coords][eptm.vert_df.is_basal==1]
        
        
        E_ab = 0.5 * (np.linalg.norm(bvert_pos - cell_pos) -
                      eptm.cell_df['prefered_hab'].values)**2
        
        basal_df = eptm.edge_df[
            ['length', 'basal_elasticity']][eptm.edge_df['is_basal' == 1]]
        E_bl = basal_df.eval('0.5 * length * basal_elasticity')
        
        

In [19]:
vp_view(eptm)